In [1]:
import torch
import torchani
import glob
import cclib
import pandas as pd

In [2]:
# path to ANI-2x subdirectories
# path = '/home/dakota/Projects/ml-benchmark/ani-2x'

In [3]:
import numpy as np
from math import sqrt
from chemreps.utils.molecule import Molecule
import re

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


In [4]:
# for ANI-2x from sae
# atom_selfE = {
# 'H': -0.5975740895940036,
# 'C': -38.08963824205658,
# 'N': -54.71039678680539,
# 'O': -75.19115551439664,
# 'S': -398.1787966261814,
# 'F': -99.79839227891235,
# 'Cl': -460.206968972609
# }

# atom_conv = {1: 0, 6: 1, 7: 2, 8: 3, 16: 4, 9: 5, 17: 6}
# atom_conv = {0: 1, 1: 6, 2: 7, 3: 6, 4: 16, 5: 9, 6: 17}

device = torch.device('cpu')
# const_file = path + '/rHCNOSFCl-5.1R_16-3.5A_a8-4.params'  # noqa: E501
# consts = torchani.neurochem.Constants(const_file)
# aev_computer = torchani.AEVComputer(**consts)
# sae_file = path + '/sae_linfit.dat'  # noqa: E501
# energy_shifter = torchani.neurochem.load_sae(sae_file)
# model_prefix = path + '/train'  # noqa: E501
# ensemble = torchani.neurochem.load_model_ensemble(consts.species, model_prefix, 8)  # noqa: E501

# model = torch.nn.Sequential(aev_computer, ensemble, energy_shifter).to(device)
model = torchani.models.ANI1x(periodic_table_index=True).to(device)

### Bond Stretch

In [5]:
results = []
for out in sorted(glob.iglob('molecules/stretch/*/sdf/*.sdf'), key=numericalSort):
    try:
        name = out.split('ch/')[1].split('/sdf')[0]
        pt = out.split('sdf/')[1].split('.')[0]
        
        if name == 'HF':
            d = {}
            d.update({'name': name})
            d.update({'point': pt})
            d.update({'ani1x_energy': np.nan})
        #         d.update({'force': list(force.cpu().data.numpy())})
            results.append(d)
        else:
            mol = Molecule(out)
            atoms = []
            for i in mol.at_num:
                atoms.append(i)

            coords = mol.xyz.tolist()

            coordinates = torch.tensor([coords],
                                   requires_grad=True, device=device)
            species = torch.tensor([atoms], device=device)

            _, energy = model((species, coordinates))

            d = {}
            d.update({'name': name})
            d.update({'point': pt})
            d.update({'ani1x_energy': energy.item()})
        #         d.update({'force': list(force.cpu().data.numpy())})
            results.append(d)

            print('Name:{}/{} \tEnergy:{} \tEnergy eV:{}'.format(name, pt, energy.item(), 27.2114*(energy.item())))
    except:
        print('File {} did not run'.format(out))

Name:C6H6-C/0 	Energy:-230.5537654295962 	Energy eV:-6273.690732610915
Name:C6H6-C/1 	Energy:-230.64476992044857 	Energy eV:-6276.167092213294
Name:C6H6-C/2 	Energy:-231.2146523729365 	Energy eV:-6291.674391580925
Name:C6H6-C/3 	Energy:-231.83408866677692 	Energy eV:-6308.530120347134
Name:C6H6-C/4 	Energy:-232.09876510402447 	Energy eV:-6315.732336751652
Name:C6H6-C/5 	Energy:-232.17841544602325 	Energy eV:-6317.899734067917
Name:C6H6-C/6 	Energy:-232.13088666130116 	Energy eV:-6316.606409295331
Name:C6H6-C/7 	Energy:-232.00559583906343 	Energy eV:-6313.1970706150905
Name:C6H6-C/8 	Energy:-231.85580368194988 	Energy eV:-6309.121016311011
Name:C6H6-C/9 	Energy:-231.63534934673717 	Energy eV:-6303.122145213804
Name:C6H6-C/10 	Energy:-231.36525858793667 	Energy eV:-6295.77259753978
Name:C6H6-C/11 	Energy:-231.1593968406718 	Energy eV:-6290.170811190257
Name:C6H6-C/12 	Energy:-231.02402855310848 	Energy eV:-6286.487250570056
Name:C6H6-C/13 	Energy:-230.9896226063769 	Energy eV:-6285.55101

Name:CH3OH/24 	Energy:-115.67118464659285 	Energy eV:-3147.5748738922966
Name:CH3OH/25 	Energy:-115.67104935336243 	Energy eV:-3147.571192374087
Name:CH3OH/26 	Energy:-115.67133048891407 	Energy eV:-3147.5788424660363
Name:CH3OH/27 	Energy:-115.67208919550609 	Energy eV:-3147.5994879345944
Name:CH3OH/28 	Energy:-115.67288299433272 	Energy eV:-3147.6210883119857
Name:CH3OH/29 	Energy:-115.67327046643328 	Energy eV:-3147.631631970303
Name:CH3OH/30 	Energy:-115.67290171298515 	Energy eV:-3147.6215976727244
Name:CH3OH/31 	Energy:-115.67219213924687 	Energy eV:-3147.6022891779025
Name:CH3OH/32 	Energy:-115.67193156077843 	Energy eV:-3147.5951984729663
Name:CH3OH/33 	Energy:-115.67217675845455 	Energy eV:-3147.6018706450104
Name:CH3OH/34 	Energy:-115.67233146696668 	Energy eV:-3147.6060804802173
Name:CH3OH/35 	Energy:-115.6718794374479 	Energy eV:-3147.59378012417
Name:CH3OH/36 	Energy:-115.6710375404669 	Energy eV:-3147.570870928661
Name:CH3OH/37 	Energy:-115.67042547061425 	Energy eV:-3147

Name:H2/16 	Energy:-1.0174020172719573 	Energy eV:-27.684933252794142
Name:H2/17 	Energy:-1.0048660577897643 	Energy eV:-27.343812244940395
Name:H2/18 	Energy:-1.0031899602775192 	Energy eV:-27.298203285095685
Name:H2/19 	Energy:-1.010200613892517 	Energy eV:-27.488972984874838
Name:H2/20 	Energy:-1.0235248149995422 	Energy eV:-27.851543150878545
Name:H2/21 	Energy:-1.0406224371556854 	Energy eV:-28.31679338641822
Name:H2/22 	Energy:-1.059415319717369 	Energy eV:-28.828174030957218
Name:H2/23 	Energy:-1.078965129113636 	Energy eV:-29.360151714362793
Name:H2/24 	Energy:-1.0991143600706672 	Energy eV:-29.908440497626955
Name:H2/25 	Energy:-1.1187284828786468 	Energy eV:-30.442168239004012
Name:H2/26 	Energy:-1.1366496251587486 	Energy eV:-30.92982761004477
Name:H2/27 	Energy:-1.152517447030983 	Energy eV:-31.36161325813889
Name:H2/28 	Energy:-1.1660443784360504 	Energy eV:-31.72969999937474
Name:H2/29 	Energy:-1.1774766629253244 	Energy eV:-32.04078846552617
Name:H2/30 	Energy:-1.1865761

Name:HCCH/24 	Energy:-77.1953521117701 	Energy eV:-2100.5936044542213
Name:HCCH/25 	Energy:-77.1935348853602 	Energy eV:-2100.5441551794906
Name:HCCH/26 	Energy:-77.19133374543328 	Energy eV:-2100.4842590804833
Name:HCCH/27 	Energy:-77.18966493458886 	Energy eV:-2100.4388484010715
Name:HCCH/28 	Energy:-77.1886769130959 	Energy eV:-2100.411962953018
Name:HCCH/29 	Energy:-77.1877901300921 	Energy eV:-2100.387832345988
Name:HCCH/30 	Energy:-77.1863906428351 	Energy eV:-2100.349750338443
Name:HCCH/31 	Energy:-77.18554671557088 	Energy eV:-2100.3267858960853
Name:HCCH/32 	Energy:-77.18602207751412 	Energy eV:-2100.3397211600677
Name:HCCH/33 	Energy:-77.18736131937642 	Energy eV:-2100.3761638060796
Name:HCCH/34 	Energy:-77.18873705418248 	Energy eV:-2100.4135994761814
Name:HCCH/35 	Energy:-77.18965757341523 	Energy eV:-2100.438648093231
Name:HCCH/36 	Energy:-77.19008836598535 	Energy eV:-2100.4503705621737
Name:HCCH/37 	Energy:-77.19009468407769 	Energy eV:-2100.450542486312
Name:HCCH/38 	En

Name:NH3/28 	Energy:-56.4636417686924 	Energy eV:-1536.4547416245964
Name:NH3/29 	Energy:-56.4797288034096 	Energy eV:-1536.8924923611
Name:NH3/30 	Energy:-56.49386296644654 	Energy eV:-1537.2771027251636
Name:NH3/31 	Energy:-56.50452374145236 	Energy eV:-1537.5671973381568
Name:NH3/32 	Energy:-56.51108242445473 	Energy eV:-1537.7456682848074
Name:NH3/33 	Energy:-56.513194927851075 	Energy eV:-1537.8031524597268
Name:NH3/34 	Energy:-56.51332373278994 	Energy eV:-1537.8066574224404
Name:NH3/35 	Energy:-56.51356841823865 	Energy eV:-1537.8133156560593
Name:NH3/36 	Energy:-56.51531611094099 	Energy eV:-1537.8608728212596
Name:NH3/37 	Energy:-56.51910037025657 	Energy eV:-1537.9638478151996
Name:NH3/38 	Energy:-56.52441627941873 	Energy eV:-1538.1085011457749
Name:NH3/39 	Energy:-56.52997155710902 	Energy eV:-1538.2596680291165
Name:NH3/40 	Energy:-56.53431859463301 	Energy eV:-1538.3779570059967
Name:NH3/41 	Energy:-56.53608943000283 	Energy eV:-1538.4261439155791
Name:NH3/42 	Energy:-56.

Name:biphenyl/7 	Energy:-463.1594863712031 	Energy eV:-12603.218047441356
Name:biphenyl/8 	Energy:-463.1409554659564 	Energy eV:-12602.713795566326
Name:biphenyl/9 	Energy:-463.11747285269087 	Energy eV:-12602.074800783712
Name:biphenyl/10 	Energy:-463.09532886288946 	Energy eV:-12601.47223181963
Name:biphenyl/11 	Energy:-463.07675209931915 	Energy eV:-12600.966732075414
Name:biphenyl/12 	Energy:-463.0605211435992 	Energy eV:-12600.525065046935
Name:biphenyl/13 	Energy:-463.040289486869 	Energy eV:-12599.97453334299
Name:biphenyl/14 	Energy:-463.00867143447397 	Energy eV:-12599.114161872045
Name:biphenyl/15 	Energy:-462.9867789699871 	Energy eV:-12598.518437263907
Name:biphenyl/16 	Energy:-462.97742055833226 	Energy eV:-12598.263781781003
Name:biphenyl/17 	Energy:-462.97999300353115 	Energy eV:-12598.333781616288
Name:biphenyl/18 	Energy:-462.9895635127265 	Energy eV:-12598.594208570206
Name:biphenyl/19 	Energy:-462.9982589679945 	Energy eV:-12598.830824081686
Name:biphenyl/20 	Energy:

Name:sucrose/33 	Energy:-1297.4704370717482 	Energy eV:-35305.98705133417
Name:sucrose/34 	Energy:-1297.4705228577332 	Energy eV:-35305.98938569092
Name:sucrose/35 	Energy:-1297.4706943849997 	Energy eV:-35305.994053187984
Name:sucrose/36 	Energy:-1297.4709276626782 	Energy eV:-35306.0004010002
Name:sucrose/37 	Energy:-1297.471214919813 	Energy eV:-35306.008217669005
Name:sucrose/38 	Energy:-1297.471485815473 	Energy eV:-35306.01558911916
Name:sucrose/39 	Energy:-1297.471638164945 	Energy eV:-35306.01973476159
Name:sucrose/40 	Energy:-1297.4716779435948 	Energy eV:-35306.02081719434
Name:sucrose/41 	Energy:-1297.4716759915427 	Energy eV:-35306.02076407627
Name:sucrose/42 	Energy:-1297.4716379563288 	Energy eV:-35306.019729084845
Name:sucrose/43 	Energy:-1297.4715267415122 	Energy eV:-35306.01670277399
Name:sucrose/44 	Energy:-1297.471372492142 	Energy eV:-35306.01250543268
Name:sucrose/45 	Energy:-1297.471277862318 	Energy eV:-35306.00993042268
Name:sucrose/46 	Energy:-1297.47127786231

In [6]:
df = pd.DataFrame(results, columns=['name', 'point', 'ani1x_energy'])
df

,name,point,ani1x_energy
0,C6H6-C,0,-230.553765
1,C6H6-C,1,-230.644770
2,C6H6-C,2,-231.214652
3,C6H6-C,3,-231.834089
4,C6H6-C,4,-232.098765
...,...,...,...
824,sucrose,42,-1297.471638
825,sucrose,43,-1297.471527
826,sucrose,44,-1297.471372
827,sucrose,45,-1297.471278


In [7]:
df.to_csv('data/ani1x-results.csv', index=False)

### Dihedral

In [8]:
results = []
for out in sorted(glob.iglob('molecules/dihedral/*/sdf/*.sdf'), key=numericalSort):
    try:
        name = out.split('ral/')[1].split('/sdf')[0]
        if name == 'sucrose' or name == 'biphenyl-twist':
            phi = np.nan
            psi = np.nan
            theta = out.split('sdf/')[1].split('.')[0].split('_')[1]
        else:
            phi = out.split('sdf/')[1].split('.')[0].split('_')[1]
            psi = out.split('sdf/')[1].split('.')[0].split('_')[2]
            theta = np.nan
        mol = Molecule(out)
        atoms = []
        for i in mol.at_num:
            atoms.append(i)
        coords = mol.xyz.tolist()

        coordinates = torch.tensor([coords],
                               requires_grad=True, device=device)
        species = torch.tensor([atoms], device=device)

        _, energy = model((species, coordinates))

        d = {}
        d.update({'name': name})
        d.update({'phi': phi})
        d.update({'psi': psi})
        d.update({'theta': theta})
        d.update({'ani1x_energy': energy.item()})
    #         d.update({'force': list(force.cpu().data.numpy())})
        results.append(d)

        print('Name:{}/{}/{}/{} \tEnergy:{} \tEnergy eV:{}'.format(name, phi, psi, theta, energy.item(), 27.2114*(energy.item())))
    except:
        print('File {} did not run'.format(out))

Name:ala-ala/0/0/nan 	Energy:-570.8963848949292 	Energy eV:-15534.889887929876
Name:ala-ala/0/20/nan 	Energy:-570.8757874370434 	Energy eV:-15534.329402264364
Name:ala-ala/0/40/nan 	Energy:-570.7672126107731 	Energy eV:-15531.37492923679
Name:ala-ala/0/60/nan 	Energy:-570.9002019912817 	Energy eV:-15534.993756465565
Name:ala-ala/0/80/nan 	Energy:-570.7715881169655 	Energy eV:-15531.493992885997
Name:ala-ala/0/100/nan 	Energy:-570.7241299078681 	Energy eV:-15530.202588574963
Name:ala-ala/0/120/nan 	Energy:-570.9084463567355 	Energy eV:-15535.218097191671
Name:ala-ala/0/140/nan 	Energy:-570.7742352784493 	Energy eV:-15531.566025855995
Name:ala-ala/0/160/nan 	Energy:-570.8682713956454 	Energy eV:-15534.124880255466
Name:ala-ala/0/180/nan 	Energy:-570.8934032768824 	Energy eV:-15534.808753928559
Name:ala-ala/0/-20/nan 	Energy:-569.9449742437222 	Energy eV:-15509.000672135622
Name:ala-ala/0/-40/nan 	Energy:-570.898673206649 	Energy eV:-15534.95215609541
Name:ala-ala/0/-60/nan 	Energy:-570.8

Name:ala-ala/100/140/nan 	Energy:-570.8892072440007 	Energy eV:-15534.6945739994
Name:ala-ala/100/160/nan 	Energy:-570.841455921612 	Energy eV:-15533.395193665354
Name:ala-ala/100/180/nan 	Energy:-570.9127599448778 	Energy eV:-15535.33547596405
Name:ala-ala/100/-20/nan 	Energy:-570.8018873111226 	Energy eV:-15532.318476377883
Name:ala-ala/100/-40/nan 	Energy:-570.9106719792702 	Energy eV:-15535.278659496715
Name:ala-ala/100/-60/nan 	Energy:-570.8330238462307 	Energy eV:-15533.165745089324
Name:ala-ala/100/-80/nan 	Energy:-570.894410029135 	Energy eV:-15534.836149066805
Name:ala-ala/100/-100/nan 	Energy:-570.9106414169886 	Energy eV:-15535.277827854245
Name:ala-ala/100/-120/nan 	Energy:-570.2115932107785 	Energy eV:-15516.255747495778
Name:ala-ala/100/-140/nan 	Energy:-570.9119855315306 	Energy eV:-15535.314403092692
Name:ala-ala/100/-160/nan 	Energy:-570.8810520172932 	Energy eV:-15534.472658863373
Name:ala-ala/100/-180/nan 	Energy:-570.8467751444199 	Energy eV:-15533.539937164867
Name

Name:ala-ala/-20/-160/nan 	Energy:-570.9040264041045 	Energy eV:-15535.097824092649
Name:ala-ala/-20/-180/nan 	Energy:-570.9148456902125 	Energy eV:-15535.392232014648
Name:ala-ala/-40/0/nan 	Energy:-570.925889939151 	Energy eV:-15535.692761490214
Name:ala-ala/-40/20/nan 	Energy:-570.9210611433604 	Energy eV:-15535.561363196437
Name:ala-ala/-40/40/nan 	Energy:-570.9209752977707 	Energy eV:-15535.55902721776
Name:ala-ala/-40/60/nan 	Energy:-570.9130184353211 	Energy eV:-15535.342509850896
Name:ala-ala/-40/80/nan 	Energy:-570.9135894031145 	Energy eV:-15535.358046683912
Name:ala-ala/-40/100/nan 	Energy:-570.9278886020996 	Energy eV:-15535.747147907176
Name:ala-ala/-40/120/nan 	Energy:-570.9215122760155 	Energy eV:-15535.57363914757
Name:ala-ala/-40/140/nan 	Energy:-570.9238873720982 	Energy eV:-15535.638268837114
Name:ala-ala/-40/160/nan 	Energy:-570.9147832841494 	Energy eV:-15535.390533858303
Name:ala-ala/-40/180/nan 	Energy:-570.9155510068753 	Energy eV:-15535.411424668486
Name:ala-al

Name:ala-ala/-140/40/nan 	Energy:-570.919324576936 	Energy eV:-15535.514108792837
Name:ala-ala/-140/60/nan 	Energy:-570.9134242237427 	Energy eV:-15535.353551921953
Name:ala-ala/-140/80/nan 	Energy:-570.9191568792679 	Energy eV:-15535.509545504512
Name:ala-ala/-140/100/nan 	Energy:-570.9289606214383 	Energy eV:-15535.776319054205
Name:ala-ala/-140/120/nan 	Energy:-570.9252690375664 	Energy eV:-15535.675865888836
Name:ala-ala/-140/140/nan 	Energy:-570.9219178409197 	Energy eV:-15535.584675136404
Name:ala-ala/-140/160/nan 	Energy:-570.9148366452076 	Energy eV:-15535.391985887403
Name:ala-ala/-140/180/nan 	Energy:-570.9159523845532 	Energy eV:-15535.42234671703
Name:ala-ala/-140/-20/nan 	Energy:-570.9245247245648 	Energy eV:-15535.655612090022
Name:ala-ala/-140/-40/nan 	Energy:-570.9157254994728 	Energy eV:-15535.416172856356
Name:ala-ala/-140/-60/nan 	Energy:-570.915329948149 	Energy eV:-15535.40540935106
Name:ala-ala/-140/-80/nan 	Energy:-570.9219423235276 	Energy eV:-15535.58534134244


Name:aspartame/40/100/nan 	Energy:-1029.4795507162905 	Energy eV:-28013.57984636127
Name:aspartame/40/120/nan 	Energy:-1029.4776000499821 	Energy eV:-28013.526766000086
Name:aspartame/40/140/nan 	Energy:-1029.4802330106593 	Energy eV:-28013.598412546256
Name:aspartame/40/160/nan 	Energy:-1029.4761155366994 	Energy eV:-28013.486370315342
Name:aspartame/40/180/nan 	Energy:-1029.4769650817013 	Energy eV:-28013.509487624207
Name:aspartame/40/-20/nan 	Energy:-1029.4777710631586 	Energy eV:-28013.531419508035
Name:aspartame/40/-40/nan 	Energy:-1029.4766615748501 	Energy eV:-28013.501228777877
Name:aspartame/40/-60/nan 	Energy:-1029.4768581658698 	Energy eV:-28013.50657829475
Name:aspartame/40/-80/nan 	Energy:-1029.4799087688423 	Energy eV:-28013.589589472474
Name:aspartame/40/-100/nan 	Energy:-1029.4774385437465 	Energy eV:-28013.522371189305
Name:aspartame/40/-120/nan 	Energy:-1029.4792829871274 	Energy eV:-28013.57256107592
Name:aspartame/40/-140/nan 	Energy:-1029.4777632325984 	Energy eV:

...

Name:aspartame/-160/-120/nan 	Energy:-1029.6048012495137 	Energy eV:-28016.988088721017
Name:aspartame/-160/-140/nan 	Energy:-1029.6038167297936 	Energy eV:-28016.961298561106
Name:aspartame/-160/-160/nan 	Energy:-1029.5986556381083 	Energy eV:-28016.82085803082
Name:aspartame/-160/-180/nan 	Energy:-1029.6079010039664 	Energy eV:-28017.07243737933
Name:aspartame/-180/0/nan 	Energy:-1029.5059048801756 	Energy eV:-28014.296980056413
Name:aspartame/-180/20/nan 	Energy:-1029.5053632676697 	Energy eV:-28014.28224202187
Name:aspartame/-180/40/nan 	Energy:-1029.507640433321 	Energy eV:-28014.344206887276
Name:aspartame/-180/60/nan 	Energy:-1029.5018771559096 	Energy eV:-28014.18738004032
Name:aspartame/-180/80/nan 	Energy:-1029.504443210373 	Energy eV:-28014.257205974744
Name:aspartame/-180/100/nan 	Energy:-1029.5065460622407 	Energy eV:-28014.314427518057
Name:aspartame/-180/120/nan 	Energy:-1029.5050579428769 	Energy eV:-28014.2739337068
Name:aspartame/-180/140/nan 	Energy:-1029.50740135909

Name:gly-gly/40/-60/nan 	Energy:-492.3050023331365 	Energy eV:-13396.30834048791
Name:gly-gly/40/-80/nan 	Energy:-492.3046870692691 	Energy eV:-13396.29976171671
Name:gly-gly/40/-100/nan 	Energy:-492.2993370308599 	Energy eV:-13396.154179681542
Name:gly-gly/40/-120/nan 	Energy:-492.29665970942585 	Energy eV:-13396.08132601707
Name:gly-gly/40/-140/nan 	Energy:-492.3087849780044 	Energy eV:-13396.411271550469
Name:gly-gly/40/-160/nan 	Energy:-492.30271622678845 	Energy eV:-13396.246132333632
Name:gly-gly/40/-180/nan 	Energy:-492.3073406680545 	Energy eV:-13396.3719698547
Name:gly-gly/60/0/nan 	Energy:-492.3037363602838 	Energy eV:-13396.273891594226
Name:gly-gly/60/20/nan 	Energy:-492.30179000041096 	Energy eV:-13396.220928417184
Name:gly-gly/60/40/nan 	Energy:-492.30637411913483 	Energy eV:-13396.345668705426
Name:gly-gly/60/60/nan 	Energy:-492.29859057209103 	Energy eV:-13396.133867493398
Name:gly-gly/60/80/nan 	Energy:-492.2971659465751 	Energy eV:-13396.095101438634
Name:gly-gly/60/1

Name:gly-gly/140/-160/nan 	Energy:-492.30424603960125 	Energy eV:-13396.287760682006
Name:gly-gly/140/-180/nan 	Energy:-492.3060561581573 	Energy eV:-13396.337016542082
Name:gly-gly/160/0/nan 	Energy:-492.30100558838456 	Energy eV:-13396.199583467767
Name:gly-gly/160/20/nan 	Energy:-492.29982078215687 	Energy eV:-13396.167343231584
Name:gly-gly/160/40/nan 	Energy:-492.30738769611924 	Energy eV:-13396.373249554179
Name:gly-gly/160/60/nan 	Energy:-492.2998094721755 	Energy eV:-13396.167035471157
Name:gly-gly/160/80/nan 	Energy:-492.30352328857987 	Energy eV:-13396.268093614863
Name:gly-gly/160/100/nan 	Energy:-492.29512132844536 	Energy eV:-13396.039464516858
Name:gly-gly/160/120/nan 	Energy:-492.2986193760356 	Energy eV:-13396.134651289056
Name:gly-gly/160/140/nan 	Energy:-492.30582577130406 	Energy eV:-13396.330747393264
Name:gly-gly/160/160/nan 	Energy:-492.30267222365944 	Energy eV:-13396.244934946888
Name:gly-gly/160/180/nan 	Energy:-492.3013465865573 	Energy eV:-13396.208862505446


Name:gly-gly/-80/160/nan 	Energy:-492.30423917016594 	Energy eV:-13396.287573755055
Name:gly-gly/-80/180/nan 	Energy:-492.31096597156136 	Energy eV:-13396.470619438545
Name:gly-gly/-80/-20/nan 	Energy:-492.3087909682712 	Energy eV:-13396.411434554015
Name:gly-gly/-80/-40/nan 	Energy:-492.3103376790008 	Energy eV:-13396.453522718362
Name:gly-gly/-80/-60/nan 	Energy:-492.30455549201577 	Energy eV:-13396.296181315438
Name:gly-gly/-80/-80/nan 	Energy:-492.30476480862706 	Energy eV:-13396.301877113476
Name:gly-gly/-80/-100/nan 	Energy:-492.3005516096076 	Energy eV:-13396.187230069678
Name:gly-gly/-80/-120/nan 	Energy:-492.30620722612946 	Energy eV:-13396.3411273131
Name:gly-gly/-80/-140/nan 	Energy:-492.3109622015676 	Energy eV:-13396.470516851736
Name:gly-gly/-80/-160/nan 	Energy:-492.30441844603627 	Energy eV:-13396.292452102472
Name:gly-gly/-80/-180/nan 	Energy:-492.30579070887177 	Energy eV:-13396.329793295394
Name:gly-gly/-100/0/nan 	Energy:-492.302161531063 	Energy eV:-13396.231038286

Name:gly-gly/-180/-120/nan 	Energy:-492.29841508111565 	Energy eV:-13396.129092138272
Name:gly-gly/-180/-140/nan 	Energy:-492.30925138434975 	Energy eV:-13396.423963120096
Name:gly-gly/-180/-160/nan 	Energy:-492.3030667617044 	Energy eV:-13396.255670879444
Name:gly-gly/-180/-180/nan 	Energy:-492.3072106554231 	Energy eV:-13396.368432028981
Name:sucrose/nan/nan/0 	Energy:-1297.618195659943 	Energy eV:-35310.00776938097
Name:sucrose/nan/nan/20 	Energy:-1297.5789185683207 	Energy eV:-35308.938984730004
Name:sucrose/nan/nan/40 	Energy:-1297.3154508802072 	Energy eV:-35301.76966008167
Name:sucrose/nan/nan/60 	Energy:-1297.2387945274786 	Energy eV:-35299.683733405036
Name:sucrose/nan/nan/80 	Energy:-1297.4572539175824 	Energy eV:-35305.6283192529
Name:sucrose/nan/nan/100 	Energy:-1296.901976175137 	Energy eV:-35290.51843449212
Name:sucrose/nan/nan/120 	Energy:-1296.5681618223862 	Energy eV:-35281.43487861368
Name:sucrose/nan/nan/140 	Energy:-1297.0812127659992 	Energy eV:-35295.395713060716


In [9]:
df = pd.DataFrame(results, columns=['name', 'phi', 'psi', 'theta', 'ani1x_energy'])
df

,name,phi,psi,theta,ani1x_energy
0,ala-ala,0,0,NaN,-570.896385
1,ala-ala,0,20,NaN,-570.875787
2,ala-ala,0,40,NaN,-570.767213
3,ala-ala,0,60,NaN,-570.900202
4,ala-ala,0,80,NaN,-570.771588
...,...,...,...,...,...
1122,sucrose,NaN,NaN,-100,-1295.307368
1123,sucrose,NaN,NaN,-120,-1294.438549
1124,sucrose,NaN,NaN,-140,-1294.348768
1125,sucrose,NaN,NaN,-160,-1296.472947


In [10]:
df.to_csv('data/ani1x-dihedral-results.csv', index=False)